## CRAG(Corrective Retrieval Augmented Generation)

> - 참고 자료: https://wikidocs.net/270686
> - 관련 논문: https://arxiv.org/pdf/2401.15884

CRAG는 RAG의 개선된 버전입니다. **RAG의 한계를 보완**하기 위해 개발된 기술로, 검색된 문서의 품질을 평가하고 필요시 수정하는 메커니즘을 추가했습니다.

### 주요 특징

**1. 검색 평가자(Retrieval Evaluator)**
- 검색된 문서의 관련성과 품질을 평가
- 문서가 충분히 정확하고 관련성이 있는지 판단

**2. 수정 메커니즘**
- 검색 결과가 부정확하거나 불충분할 경우 대응
- 추가 검색, 웹 검색, 또는 지식 정제 수행

**3. 동적 처리 방식**
검색 품질에 따라 세 가지 경로로 분기:
- **Correct**: 검색 결과가 좋을 경우 → 그대로 사용
- **Incorrect**: 검색 결과가 나쁠 경우 → 웹 검색 등 추가 소스 활용
- **Ambiguous**: 애매한 경우 → 지식 정제 후 사용

### RAG와의 차이점

- **RAG**: 검색된 문서를 무조건 신뢰하고 사용
- **CRAG**: 검색 품질을 평가하고, 필요시 보완하거나 다른 소스 활용

이를 통해 더 정확하고 신뢰할 수 있는 답변 생성이 가능합니다.

## 환경 변수

In [ ]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## 문서 다운로드

In [8]:
import os
import urllib.request


def download_file(url, dir="./temp"):
    if not os.path.exists(dir):
        os.makedirs(dir)

    file_path = os.path.join(dir, "doc.pdf")
    urllib.request.urlretrieve(url, file_path)

    file_size = os.path.getsize(file_path)
    print(f"다운로드 완료! 파일 크기: {file_size:,}", "bytes")

    return file_path

In [9]:
# "소프트웨어정책연구소(SPRi) - 2025년 10월호" PDF를 다운로드합니다.
downloaded_path = download_file("https://spri.kr/download/23735")
downloaded_path

다운로드 완료! 파일 크기: 1,207,479 bytes


'./temp/doc.pdf'

## 문서 로더

In [10]:
from langchain_community.document_loaders import PyPDFLoader

documents = PyPDFLoader(downloaded_path).load()
print(f"총 {len(documents)} 페이지 로드됨")

총 29 페이지 로드됨


## 문서 분할

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splits = text_splitter.split_documents(documents)
print(f"총 {len(splits)}개의 청크로 분할됨")

총 70개의 청크로 분할됨


## 문서 임베딩

In [12]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS


# 임베딩 모델 생성
underlying_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    document_embedding_cache=LocalFileStore("./cache"),
    namespace=underlying_embeddings.model,
    query_embedding_cache=True,
)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.from_documents(splits, embeddings)


/Users/anpigon/Workspace/Wantedlab/langchain-academy/venv/lib/python3.12/site-packages/langchain/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


## 리트리버 생성

In [13]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 50},
)

In [14]:
retriever.invoke("미국 캘리포니아주 의회가 발의한 법안은?")

[Document(id='625c5d38-8e8f-49b7-9ab2-53cf8cf54534', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-10-02T12:57:42+09:00', 'author': 'dj', 'moddate': '2025-10-02T12:57:42+09:00', 'pdfversion': '1.4', 'source': './temp/doc.pdf', 'total_pages': 29, 'page': 6, 'page_label': '7'}, page_content='정책･법제기업･산업기술･연구인력･교육\n5\n미국 캘리포니아주 의회, 미국 최초로 미성년자 보호를 위한 AI 챗봇 규제 통과n미국 캘리포니아주 의회가 미국 최초로 미성년자와 취약 사용자 보호를 위해 AI 동반자 챗봇을 규제하는 SB-243 법안을 통과시켰으며, 주지사 서명 시 2026년 1월 1일부터 발효 예정n동 법안은 동반자 챗봇 운영 플랫폼을 대상으로 사용자의 자살이나 자해를 방지하기 위한 프로토콜 수립 및 미성년자 보호 조치를 요구하고, 위법으로 인한 피해자에게 민사소송 권리를 부여\nKEY Contents'),
 Document(id='5fc0f45e-99cd-4944-a171-e9a7add59965', metadata={'producer': 'Hancom PDF 1.3.0.505', 'creator': 'Hancom PDF 1.3.0.505', 'creationdate': '2025-10-02T12:57:42+09:00', 'author': 'dj', 'moddate': '2025-10-02T12:57:42+09:00', 'pdfversion': '1.4', 'source': './temp/doc.pdf', 'total_pages': 29, 'page': 9, 'page_label': '10'}, page_content='<‘제미나이 2.

## 관련성 평가 노드

In [ ]:
from pydantic import BaseModel


class GradeDocuments(BaseModel):
    """검색된 문서의 관련성을 판단하기 위한 binary score"""